In [1]:
import pandas as pd
import requests
import re
import os
from tqdm import tqdm

In [2]:
# import tarfile
# # Specify the path to your .tar file
# tar_path = "IGWN-GWOSC-snapshot-2023-11-04.tar"
# 
# # Open and list the contents of the .tar file
# with tarfile.open(tar_path, 'r') as tar:
#     file_names = tar.getnames()
#     print("Files in tar archive:", file_names)
# 
#     # Extract all files or a specific file
#     tar.extractall()  # Extracts to the current directory

In [3]:
filepath = "snapshot-2023-11-04/GWTC/GWTC.json"
df = pd.read_json(filepath)
events = list(df.index)

In [4]:
N_events = 0
ignore_list = []

for i in tqdm(range(len(events))):
    event = events[i]
    # read file with name event
    event_file = f"snapshot-2023-11-04/GWTC/{event}.json"
    event_df = pd.read_json(event_file)
    
    key_val = list(event_df["events"][f"{event}"]["parameters"].keys())[0]
    url = event_df["events"][f"{event}"]["parameters"][key_val]["data_url"]
    
    if event_df["events"][f"{event}"]["far"] > 1:
        ignore_list.append(event)
        continue
    
    if "https" not in url:
        for key_val in event_df["events"][f"{event}"]["parameters"].keys():
            url = event_df["events"][f"{event}"]["parameters"][key_val]["data_url"]
            if "https" in url:
                break
    
    pattern1= r'([^/]+)/content$'
    pattern2 = r'(?<=/)([^/]+\.h5)'
    pattern3 = r'(?<=/)([^/]+\.hdf5)'
    match1 = re.search(pattern1, url)
    match2 = re.search(pattern2, url)
    match3 = re.search(pattern3, url)
    
    if match1:
        filename = match1.group(1)
    elif match2:
        filename = match2.group(1)
    elif match3:
        filename = match3.group(1)
    elif i in [44, 73, 75]:
        continue # GW190909_114149, GW190424_180648 and GW190426_152155 has no PE for GWTC2.1 so we skip it. 
    else:
        print(f"No match found for {event} at i = {i}")
        break
    
    file_path = f"../{filename}"  # Change the filename as needed
    
    N_events += 1
    
    if os.path.exists(file_path):
        continue
    
    with open(file_path, "wb") as file:  # Use 'wb' to write binary content
        file.write(requests.get(url, allow_redirects=True).content)
print(f"Downloaded {N_events} events.")

100%|██████████| 93/93 [00:00<00:00, 1280.76it/s]

Downloaded 74 events.


In [5]:
# i = 84
# event = events[i]
# # read file with name event
# event_file = f"snapshot-2023-11-04/GWTC/{event}.json"
# event_df = pd.read_json(event_file)
# key_val = list(event_df["events"][f"{event}"]["parameters"].keys())[1]
# url = event_df["events"][f"{event}"]["parameters"][key_val]["data_url"]
# 
# pattern1= r'([^/]+)/content$'
# pattern2 = r'(?<=/)([^/]+\.h5)'
# pattern3 = r'(?<=/)([^/]+\.hdf5)'
# match1 = re.search(pattern1, url)
# match2 = re.search(pattern2, url)
# match3 = re.search(pattern3, url)
# 
# if match1:
#     filename = match1.group(1)
# elif match2:
#     filename = match2.group(1)
# elif match3:
#     filename = match3.group(1)
# else:
#     print(f"No match found for {event} at i = {i}")
# 
# file_path = f"../{filename}"  # Change the filename as needed
# file_path